# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [95]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [96]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [97]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [98]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [99]:
def foldl2(f, x0, lst):
    return foldr(lambda x, func: lambda y: func(f(y, x)), lambda x: x, lst)(x0)

In [100]:
def foldr2(f, x0, lst):
    return foldl(lambda func, x: lambda y: func(f(x, y)), lambda x: x, lst)(x0)

In [101]:
foldl2(f,1,[1,2,3]), \
foldr2(f,1,[1,2,3])

(0.16666666666666666, 1.5)

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [102]:
def check_inv(a, b):
    b_copy = list(b)
    spaces = a.count(" ")
    for i, _ in enumerate(a):
        if a[i] in b_copy:
            b_copy.remove(a[i])
            if not b_copy:
                return True
        else:
            b_copy = list(b)
    return False

In [103]:
check_inv('abcrotm', 'tro')

True

Сложность: O(len(a)*len(b))

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [104]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        nodes_list = list()
        def iter_tree(node):
            if node.left:
                iter_tree(node.left)
            if node.right:
                iter_tree(node.right)
            if not node.right and not node.left:
                nodes_list.append(node.value)
        iter_tree(self)
        return iter(nodes_list)
    
    
    def __str__(self):
        tree_str = "(" + str(self.value)
        if self.left:
            tree_str += str(self.left)
        if self.right:
            tree_str += str(self.right)
        return tree_str  + ")"
    
    
    def __repr__(self):
        repr_str = "Tree(" + str(self.value)
        if self.left:
            repr_str += ", " + repr(self.left)
        if self.right:
            repr_str += ", " + repr(self.right)
        return repr_str  + ")"
    
    

In [105]:
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

In [106]:
list(tree) == [3, 4, 2]

True

# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [107]:
import re

In [108]:
def check_if_num(expr):
    return not any(math_symb in expr for math_symb in [' + ', ' - ', ' * ', ' / '])

def calc(expr, negative=False): 
    expr = expr.replace("(-", "(0 - ")
    if check_if_num(expr):
        return float(expr)
    if "(" in expr:
        list_expr = re.split('\(|\)', expr)
        while len(list_expr) > 1:
            for i, el in enumerate(list_expr):
                if len(el) > 0:
                    if el.strip()[-1] not in ['+', '-', '*', '/'] and el.strip()[0] not in ['+', '-', '*', '/']:
                        list_expr[i] = str(calc(list_expr[i]))
                        list_expr[i-1:i+2] = [''.join(list_expr[i-1:i+2])]
        expr = ''.join(list_expr)
    if ' + ' in expr:
        result = 0
        operator_split = expr.split(' + ')
        for el in operator_split:
            result += calc(el)
        return float(result)
    if ' - ' in expr:
        operator_split = expr.split(' - ')
        result = calc(operator_split[0])
        for el in operator_split[1:]:
            result -= calc(el)
        return float(result)
    if ' / ' in expr:
        operator_split = expr.split(' / ')
        result = calc(operator_split[0])
        for el in operator_split[1:]:
            result /= calc(el)
        return float(result)
    if ' * ' in expr:
        result = 1
        operator_split = expr.split(' * ')
        for el in operator_split:
            result *= calc(el)
        return float(result)

In [109]:
calc('2 * (15 - 3 * 4) - 2') == 4

True